In [ ]:
import yfinance as yf
import pandas as pd
import ta
import time

# Baixar os últimos 30 dias de dados da Apple
ticker = "AAPL"
def coletar_dados():
    dados = yf.download(ticker, period="30d", interval="1d")

    # Garantir que "Close" seja uma série 1D
    dados["Close"] = dados["Close"].astype(float).squeeze()

    # Calcular a Média Móvel Simples (SMA) de 10 e 20 dias
    dados["SMA_10"] = dados["Close"].rolling(window=10).mean()
    dados["SMA_20"] = dados["Close"].rolling(window=20).mean()

    # Calcular o RSI (Relative Strength Index) de 14 dias
    rsi_indicator = ta.momentum.RSIIndicator(close=dados["Close"].squeeze(), window=14)
    dados["RSI"] = rsi_indicator.rsi()

    # Calcular as Bandas de Bollinger (20 dias)
    bollinger = ta.volatility.BollingerBands(close=dados["Close"].squeeze(), window=20)
    dados["Bollinger_High"] = bollinger.bollinger_hband()
    dados["Bollinger_Low"] = bollinger.bollinger_lband()
    return dados

def gerar_recomendacao(df):
    if len(df) < 20:
        return "Dados insuficientes para análise."

    ult_linha = df.iloc[-1]  # Última linha corretamente indexada
    recomendacoes = []

    # 🔹 Verificando se "RSI" existe e não é NaN antes da análise
    if "RSI" in df.columns:
        rsi_valor = ult_linha["RSI"].values[0] if isinstance(ult_linha["RSI"], pd.Series) else ult_linha["RSI"]
        print("Última linha:", ult_linha)  # Debugging: print the last row
        print("Valor do RSI:", rsi_valor)  # Debugging: print the RSI value
        
        if isinstance(rsi_valor, (float, int)) and not pd.isna(rsi_valor):  # Verificando se o valor não é NaN
            if rsi_valor < 30:
                recomendacoes.append("RSI indica COMPRA (ativo sobrevendido).")
            elif rsi_valor > 70:
                recomendacoes.append("RSI indica VENDA (ativo sobrecomprado).")
            
    # 🔹 Verificando cruzamento de médias móveis
    if len(df) > 1:
        prev_linha = df.iloc[-2]
        sma_10_prev = prev_linha["SMA_10"].values[0] if isinstance(prev_linha["SMA_10"], pd.Series) else prev_linha["SMA_10"]
        sma_20_prev = prev_linha["SMA_20"].values[0] if isinstance(prev_linha["SMA_20"], pd.Series) else prev_linha["SMA_20"]
        sma_10_ult = ult_linha["SMA_10"].values[0] if isinstance(ult_linha["SMA_10"], pd.Series) else ult_linha["SMA_10"]
        sma_20_ult = ult_linha["SMA_20"].values[0] if isinstance(ult_linha["SMA_20"], pd.Series) else ult_linha["SMA_20"]

        if sma_10_prev < sma_20_prev and sma_10_ult > sma_20_ult:
            recomendacoes.append("Cruzamento de Médias Móveis indica COMPRA.")
        elif sma_10_prev > sma_20_prev and sma_10_ult < sma_20_ult:
            recomendacoes.append("Cruzamento de Médias Móveis indica VENDA.")

    # 🔹 Analisando Bandas de Bollinger
    if "Close" in df.columns and "Bollinger_Low" in df.columns and "Bollinger_High" in df.columns:
        close_valor = ult_linha["Close"].values[0] if isinstance(ult_linha["Close"], pd.Series) else ult_linha["Close"]
        bollinger_low = ult_linha["Bollinger_Low"].values[0] if isinstance(ult_linha["Bollinger_Low"], pd.Series) else ult_linha["Bollinger_Low"]
        bollinger_high = ult_linha["Bollinger_High"].values[0] if isinstance(ult_linha["Bollinger_High"], pd.Series) else ult_linha["Bollinger_High"]

        if close_valor < bollinger_low:
            recomendacoes.append("Banda de Bollinger indica COMPRA (preço abaixo da banda inferior).")
        elif close_valor > bollinger_high:
            recomendacoes.append("Banda de Bollinger indica VENDA (preço acima da banda superior).")
    
    return "\n".join(recomendacoes) if recomendacoes else "Nenhuma recomendação clara no momento."

# Supondo que 'dados' seja o seu DataFrame
while True:
    print("\n🔄 Atualizando análise...")
    dados = coletar_dados()
    
    if dados is not None:
        recomendacao = gerar_recomendacao(dados)
        print("\n📊 Recomendação baseada nos indicadores:")
        print(recomendacao)
    else:
        print("⚠ Erro ao coletar dados. Tentando novamente...")

    time.sleep(60)  # Aguarda 60 segundos antes da próxima análise


🔄 Atualizando análise...


[*********************100%***********************]  1 of 1 completed


Última linha: Price           Ticker
Close           AAPL      2.328000e+02
High            AAPL      2.331300e+02
Low             AAPL      2.266500e+02
Open            AAPL      2.272000e+02
Volume          AAPL      4.448913e+07
SMA_10                    2.312150e+02
SMA_20                    2.332670e+02
RSI                       4.544721e+01
Bollinger_High            2.467558e+02
Bollinger_Low             2.197782e+02
Name: 2025-02-04 00:00:00, dtype: float64
Valor do RSI: 45.447213548750966

📊 Recomendação baseada nos indicadores:
Nenhuma recomendação clara no momento.

🔄 Atualizando análise...


[*********************100%***********************]  1 of 1 completed


Última linha: Price           Ticker
Close           AAPL      2.328000e+02
High            AAPL      2.331300e+02
Low             AAPL      2.266500e+02
Open            AAPL      2.272000e+02
Volume          AAPL      4.448913e+07
SMA_10                    2.312150e+02
SMA_20                    2.332670e+02
RSI                       4.544721e+01
Bollinger_High            2.467558e+02
Bollinger_Low             2.197782e+02
Name: 2025-02-04 00:00:00, dtype: float64
Valor do RSI: 45.447213548750966

📊 Recomendação baseada nos indicadores:
Nenhuma recomendação clara no momento.
